In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd
from IPython.display import clear_output

# Função para carregar e transformar os datasets de regressão em problemas binários
def carregar_dataset_regressao_binario(nome_dataset):
    if nome_dataset == 'california_housing_binary':
        data = fetch_openml(name='CaliforniaHousing', version=1)
        X = pd.DataFrame(data.data, columns=data.feature_names)
        y = pd.qcut(data.target, q=2, labels=[0, 1]).astype(int)  # Dividindo mediana
        class_names = ['Low Price', 'High Price']

    elif nome_dataset == 'bike_sharing_binary':
        data = fetch_openml(name='Bike_Sharing_Demand', version=2)
        X = data.data.select_dtypes(include='number')  # Apenas variáveis numéricas
        y = pd.qcut(data.target, q=2, labels=[0, 1]).astype(int)
        class_names = ['Low Demand', 'High Demand']

    elif nome_dataset == 'mercedes_binary':
        data = fetch_openml(name='Mercedes_Benz_Greener_Manufacturing', version=1)
        X = data.data.select_dtypes(include='number')
        y = pd.qcut(data.target, q=2, labels=[0, 1]).astype(int)
        class_names = ['Low Y', 'High Y']

    elif nome_dataset == 'superconductivity_binary':
        data = fetch_openml(name='Superconduct', version=1)
        X = pd.DataFrame(data.data)
        y = pd.qcut(data.target, q=2, labels=[0, 1]).astype(int)
        class_names = ['Low Critical Temp', 'High Critical Temp']

    elif nome_dataset == 'concrete_strength_binary':
        data = fetch_openml(name='Concrete_Compressive_Strength', version=1)
        X = pd.DataFrame(data.data)
        y = pd.qcut(data.target, q=2, labels=[0, 1]).astype(int)
        class_names = ['Low Strength', 'High Strength']

    elif nome_dataset == 'house_sales_binary':
        data = fetch_openml(name='House_sales', version=1)
        X = data.data.select_dtypes(include='number')
        y = pd.qcut(data.target, q=2, labels=[0, 1]).astype(int)
        class_names = ['Low Price', 'High Price']

    elif nome_dataset == 'energy_efficiency_binary':
        data = fetch_openml(name='Energy_efficiency', version=2)
        X = data.data.select_dtypes(include='number')
        y = pd.qcut(data.target, q=2, labels=[0, 1]).astype(int)
        class_names = ['Low Efficiency', 'High Efficiency']

    elif nome_dataset == 'student_performance_binary':
        data = fetch_openml(name='StudentPerformance', version=1)
        X = data.data.select_dtypes(include='number')
        y = pd.qcut(data.target, q=2, labels=[0, 1]).astype(int)
        class_names = ['Low Performance', 'High Performance']

    elif nome_dataset == 'insurance_binary':
        data = fetch_openml(name='Insurance', version=1)
        X = data.data.select_dtypes(include='number')
        y = pd.qcut(data.target, q=2, labels=[0, 1]).astype(int)
        class_names = ['Low Cost', 'High Cost']

    elif nome_dataset == 'communities_crime_binary':
        data = fetch_openml(name='communities', version=1)
        X = data.data.select_dtypes(include='number')
        y = pd.qcut(data.target, q=2, labels=[0, 1]).astype(int)
        class_names = ['Low Crime', 'High Crime']

    else:
        raise ValueError("Nome do dataset não reconhecido. Escolha um dataset válido.")
    
    return X, y, class_names

# Menu de seleção de datasets de regressão
menu = '''
|  ************************* MENU ***************************  |
|  0 - california_housing_binary   |  1 - bike_sharing_binary    |
|  2 - mercedes_binary             |  3 - superconductivity_binary|
|  4 - concrete_strength_binary    |  5 - house_sales_binary      |
|  6 - energy_efficiency_binary    |  7 - student_performance_binary|
|  8 - insurance_binary            |  9 - communities_crime_binary |
|  Q - SAIR                                                   |
|-------------------------------------------------------------|
'''

# Loop do menu com validação adequada
while True:
    print(menu)
    opcao = input("Digite o número do dataset ou 'Q' para sair: ").upper().strip()

    if opcao == 'Q':
        clear_output()
        print("Você escolheu sair.")
        break
    elif opcao.isdigit() and 0 <= int(opcao) <= 9:
        nomes_datasets = [
            'california_housing_binary', 'bike_sharing_binary', 'mercedes_binary', 'superconductivity_binary',
            'concrete_strength_binary', 'house_sales_binary', 'energy_efficiency_binary', 'student_performance_binary',
            'insurance_binary', 'communities_crime_binary'
        ]
        nome_dataset = nomes_datasets[int(opcao)]
        clear_output()
        print(f"Dataset '{nome_dataset}' escolhido.")
        try:
            X, y, class_names = carregar_dataset_regressao_binario(nome_dataset)
            print(f"Dataset {nome_dataset} carregado com sucesso.")
            print("Classes:", class_names)
            print("Amostras:", X.shape[0], "| Atributos:", X.shape[1])
            break
        except Exception as e:
            print(f"Erro ao processar o dataset: {e}")
    else:
        clear_output()
        print("Opção inválida. Por favor, escolha um número do menu ou 'Q' para sair.")


Dataset 'concrete_strength_binary' escolhido.


In [ ]:
# DATASETS CLASSIFICACÁO BINÁRIOS
from sklearn.datasets import load_iris, load_wine, load_breast_cancer, load_digits, load_diabetes, fetch_california_housing, fetch_openml
import pandas as pd
from IPython.display import clear_output

# Função para carregar os datasets
def carregar_dataset(nome_dataset):
    if nome_dataset == 'iris':
        data = load_iris()
        X, y = pd.DataFrame(data.data, columns=data.feature_names), data.target
        class_names = data.target_names
    
    elif nome_dataset == 'wine':
        data = load_wine()
        X, y = pd.DataFrame(data.data, columns=data.feature_names), data.target
        class_names = data.target_names
    
    elif nome_dataset == 'breast_cancer':
        data = load_breast_cancer()
        X, y = pd.DataFrame(data.data, columns=data.feature_names), data.target
        class_names = data.target_names
    
    elif nome_dataset == 'digits':
        data = load_digits()
        X, y = pd.DataFrame(data.data, columns=[f"pixel_{i}" for i in range(data.data.shape[1])]), data.target
        class_names = [str(i) for i in range(10)]
    
    elif nome_dataset == 'diabetes':
        data = load_diabetes()
        X, y = pd.DataFrame(data.data, columns=data.feature_names), data.target
        class_names = ['Progression']  # Dados de regressão, podem ser classificados em faixas
    
    elif nome_dataset == 'boston_housing':
        data = fetch_openml(name='boston', version=1)
        X, y = pd.DataFrame(data.data, columns=data.feature_names), data.target
        class_names = ['Price']  # Regressão, mas pode ser categorizada
    
    elif nome_dataset == 'california_housing':
        data = fetch_california_housing()
        X, y = pd.DataFrame(data.data, columns=data.feature_names), data.target
        class_names = ['Price']
    
    elif nome_dataset == 'wine_quality':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=';')
        X = data.drop(columns=['quality'])
        y = data['quality']
        class_names = sorted(y.unique().tolist())
    
    elif nome_dataset == 'heart_disease':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data", header=None, na_values="?")
        data = data.dropna()  # Remove missing values
        X = data.iloc[:, :-1]
        y = data.iloc[:, -1]
        class_names = sorted(y.unique().tolist())
    
    elif nome_dataset == 'banknote_authentication':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt", header=None)
        X = data.iloc[:, :-1]
        y = data.iloc[:, -1]
        class_names = ['Legitimate', 'Forgery']
    
    elif nome_dataset == 'parkinsons':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data")
        X = data.drop(columns=['status', 'name'])
        y = data['status']
        class_names = ['Healthy', 'Parkinsons']
    
    else:
        raise ValueError("Nome do dataset não reconhecido. Escolha um dataset válido.")
    
    return X, y, class_names

# Exemplo: escolha o dataset desejado
nome_dataset = 'iris'  # Modifique para o dataset desejado
X, y, class_names = carregar_dataset(nome_dataset)
print(f"Dataset {nome_dataset} carregado com sucesso.")
print("Classes:", class_names)
print("Amostras:", X.shape[0], "| Atributos:", X.shape[1])


Dataset iris carregado com sucesso.
Classes: ['setosa' 'versicolor' 'virginica']
Amostras: 150 | Atributos: 4


In [60]:

import os
#print( '\n|  ************************* MENU **************************  |\
#        \n|  1 - iris                     |  2 - wine                    |\
#        \n|  3 - breats_cancer            |  4 - digits                 |\
#        \n|  5 - diabetes                 |  6 - boston_housing         |\
#        \n|  7 - california_housing       |  8 - wine_quality           |\
#        \n|  9 - heart_disease            |  0 - nknorth_authentication |\
#        \n|-------------------------------------------------------------|\
#        \n| Digite o Nº ->->->->-> : ')
from IPython.display import clear_output
print('\n|  ************************* MENU **************************  |\
        \n|  1 - iris                     |  2 - wine                    |\
        \n|  3 - breats_cancer            |  4 - digits                 |\
        \n|  5 - diabetes                 |  6 - boston_housing         |\
        \n|  7 - california_housing       |  8 - wine_quality           |\
        \n|  9 - heart_disease            |  0 - north_authentication   |\
        \n|-------------------------------------------------------------|\
        \n| Digite o Nº ->->->->-> : ')

clear_output()  # Limpa a saída da célula atual

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd

# Função para transformar o dataset em um problema binário (classe 0 contra as outras)
def transformar_problema_binario(y, classe_0):
    return [1 if label == classe_0 else 0 for label in y]

def analisar_instancias(X, y, class_names, classe_0=0, instancia_para_analisar=None):
    global TUDO
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])

    # Transforma o problema em binário
    y_binario = transformar_problema_binario(y, classe_0)

    # Divide o dataset em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y_binario, test_size=0.2, random_state=42)
    
    # Treina o modelo
    modelo = LogisticRegression(max_iter=200)
    modelo.fit(X_train, y_train)

    # Obtém os nomes das features
    feature_names = X.columns.tolist()  
    
    # Seleciona as instâncias para análise
    num_instancias = len(X_test)
    instancias_para_analisar = range(num_instancias) if instancia_para_analisar is None else [instancia_para_analisar]
    
    TUDO = []
    # Loop para analisar cada instância selecionada
    for idx in instancias_para_analisar:
        Vs = X_test.iloc[idx].to_dict()
        instancia_test = X_test.iloc[[idx]]

        # Calcula `gamma_A` usando `decision_function`
        gamma_A = modelo.decision_function(instancia_test)[0]
        
        # Cálculo do valor delta para cada feature
        delta = []
        w = modelo.coef_[0]
        for i, feature in enumerate(feature_names):
            if w[i] < 0:
                delta.append((Vs[feature] - X[feature].max()) * w[i])
            else:
                delta.append((Vs[feature] - X[feature].min()) * w[i])

        # Calcula R
        R = sum(delta) - gamma_A
        
        # Computa a PI-explicação para a instância atual usando nomes das features
        Xpl = one_explanation(Vs, delta, R, feature_names, modelo, instancia_test, X)
        # Imprime os resultados
        classe_verdadeira = y_test[idx]
        print(f"\nInstância {idx}:")
        print(f"Classe verdadeira (binária): {classe_verdadeira}")
        print(f"PI-Explicação: ")
        
        TUDO.append(Xpl)

        for item in Xpl:
            print(f"- {item}")
            
            
        if not Xpl:
            print('_No-PI-explanation_'*3)
        

# Função para calcular a PI-explicação e incluir os intervalos de valores mínimos e máximos que garantem a classe
def one_explanation(Vs, delta, R, feature_names, modelo, instancia_test, X):
    Xpl = []
    delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True)
    R_atual = R
    Idx = 0
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]
        feature = feature_names[sorted_idx]
        feature_value = Vs[feature]

        # Encontra os valores mínimo e máximo para manter a classe usando perturbação
        #min_val, max_val = encontrar_intervalo_perturbacao(modelo, instancia_test, feature, feature_value, classe_desejada=1, X=X)

        # Adiciona a feature com o valor da instância e o intervalo mínimo/máximo que mantém a classe
        Xpl.append(f"{feature} - {feature_value} ")
        R_atual -= delta_value
        Idx += 1
    
    return Xpl

# Função para encontrar o intervalo de perturbação para manter a classe, considerando limites de X
def encontrar_intervalo_perturbacao(modelo, instancia, feature, valor_original, classe_desejada, X, passo=0.1, max_iter=50):
    # Define os valores mínimo e máximo baseados nos dados de entrada
    min_val_data = X[feature].min()
    max_val_data = X[feature].max()
    
    # Inicializa os valores mínimo e máximo com o valor da instância
    min_val, max_val = valor_original, valor_original
    
    # Perturba negativamente
    for _ in range(max_iter):
        min_val -= passo
        if min_val < min_val_data:
            min_val = min_val_data
            break
        instancia_perturbada = instancia.copy()
        instancia_perturbada[feature] = min_val
        predicao = modelo.predict(instancia_perturbada)
        if predicao[0] != classe_desejada:
            min_val += passo
            break

    # Perturba positivamente
    for _ in range(max_iter):
        max_val += passo
        if max_val > max_val_data:
            max_val = max_val_data
            break
        instancia_perturbada = instancia.copy()
        instancia_perturbada[feature] = max_val
        predicao = modelo.predict(instancia_perturbada)
        if predicao[0] != classe_desejada:
            max_val -= passo
            break

    return min_val, max_val

# Exemplo de uso (substitua X e y pelos dados adequados)
analisar_instancias(X, y, class_names, classe_0=0)
# Crie um dicionário para armazenar a contagem de cada feature
contagem_features = {}

# Itere sobre cada item da lista TUDO
for item in TUDO:
    # Verifique se o item é uma lista
    if isinstance(item, list):
        # Itere sobre cada item da lista
        for feature in item:
            # Extraia o nome da feature
            nome_feature = feature.split(" - ")[0]

            # Verifique se a feature já está no dicionário
            if nome_feature in contagem_features:
                # Incremente a contagem
                contagem_features[nome_feature] += 1
            else:
                # Adicione a feature ao dicionário com contagem 1
                contagem_features[nome_feature] = 1

# Imprima as features e suas contagens
for nome_feature, contagem in contagem_features.items():
    print(f"Feature: {nome_feature} Contagem: {contagem}")


Instância 0:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal length (cm) - 4.7 
- petal width (cm) - 1.2 
- sepal length (cm) - 6.1 
- sepal width (cm) - 2.8 

Instância 1:
Classe verdadeira (binária): 1
PI-Explicação: 
- petal length (cm) - 1.7 
- petal width (cm) - 0.3 

Instância 2:
Classe verdadeira (binária): 0
PI-Explicação: 
- sepal width (cm) - 2.6 
- petal width (cm) - 2.3 
- sepal length (cm) - 7.7 
- petal length (cm) - 6.9 

Instância 3:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal length (cm) - 4.5 
- petal width (cm) - 1.5 
- sepal length (cm) - 6.0 
- sepal width (cm) - 2.9 

Instância 4:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal length (cm) - 4.8 
- petal width (cm) - 1.4 
- sepal width (cm) - 2.8 
- sepal length (cm) - 6.8 

Instância 5:
Classe verdadeira (binária): 1
PI-Explicação: 
- petal length (cm) - 1.5 
- petal width (cm) - 0.4 

Instância 6:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal length (cm) - 3.6 
- petal width (c

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd

# Função para transformar o dataset em um problema binário (classe 0 contra as outras)
def transformar_problema_binario(y, classe_pos):
    return [1 if label == classe_pos else 0 for label in y]

def analisar_instancias(X, y, class_names, classe_pos=0, instancia_para_analisar=None):
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])

    # Transforma o problema em binário
    y_binario = transformar_problema_binario(y, classe_pos)

    # Divide o dataset em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y_binario, test_size=0.2, random_state=42)
    
    # Treina o modelo
    modelo = LogisticRegression(max_iter=200)
    modelo.fit(X_train, y_train)

    # Obtém os nomes das features
    feature_names = X.columns.tolist()  
    
    # Seleciona as instâncias para análise
    num_instancias = len(X_test)
    instancias_para_analisar = range(num_instancias) if instancia_para_analisar is None else [instancia_para_analisar]
    
    # Loop para analisar cada instância selecionada
    for idx in instancias_para_analisar:
        Vs = X_test.iloc[idx].to_dict()
        instancia_test = X_test.iloc[[idx]]

        # Calcula `gamma_A` usando `decision_function`
        gamma_A = modelo.decision_function(instancia_test)[0]
        
        # Cálculo do valor delta para cada feature
        delta = []
        w = modelo.coef_[0]
        for i, feature in enumerate(feature_names):
            if w[i] < 0:
                delta.append((Vs[feature] - X[feature].max()) * w[i])
            else:
                delta.append((Vs[feature] - X[feature].min()) * w[i])

        # Calcula R
        R = sum(delta) - gamma_A
        
        # Computa a PI-explicação para a instância atual usando nomes das features
        Xpl = one_explanation(Vs, delta, R, feature_names, X)

        # Imprime os resultados
        classe_verdadeira = y_test[idx]
        print(f"\nInstância {idx}:")
        print(f"Classe verdadeira (binária): {classe_verdadeira}")
        print(f"PI-Explicação: ")
        for item in Xpl:
            print(f"- {item}")
        if not Xpl:
            print('_No-PI-explanation_'*3)

# Função para calcular a PI-explicação e incluir os intervalos de valores mínimos e máximos
def one_explanation(Vs, delta, R, feature_names, X):
    Xpl = []
    delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True)
    R_atual = R
    Idx = 0
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]
        feature = feature_names[sorted_idx]
        feature_value = Vs[feature]
        min_val = X[feature].min()
        max_val = X[feature].max()
        
        # Adiciona a feature com o valor da instância e o intervalo mínimo/máximo que explica a classe
        Xpl.append(f"{feature} - {feature_value} (Min: {min_val}, Max: {max_val})")
        R_atual -= delta_value
        Idx += 1
    
    return Xpl

# Executar a análise de PI-explicação no conjunto de teste do dataset selecionado
# Certifique-se de que X seja um DataFrame com colunas nomeadas
analisar_instancias(X, y, class_names, classe_pos=0)



Instância 0:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal length (cm) - 4.7 (Min: 1.0, Max: 6.9)
- petal width (cm) - 1.2 (Min: 0.1, Max: 2.5)
- sepal length (cm) - 6.1 (Min: 4.3, Max: 7.9)
- sepal width (cm) - 2.8 (Min: 2.0, Max: 4.4)

Instância 1:
Classe verdadeira (binária): 1
PI-Explicação: 
- petal length (cm) - 1.7 (Min: 1.0, Max: 6.9)
- petal width (cm) - 0.3 (Min: 0.1, Max: 2.5)

Instância 2:
Classe verdadeira (binária): 0
PI-Explicação: 
- sepal width (cm) - 2.6 (Min: 2.0, Max: 4.4)
- petal width (cm) - 2.3 (Min: 0.1, Max: 2.5)
- sepal length (cm) - 7.7 (Min: 4.3, Max: 7.9)
- petal length (cm) - 6.9 (Min: 1.0, Max: 6.9)

Instância 3:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal length (cm) - 4.5 (Min: 1.0, Max: 6.9)
- petal width (cm) - 1.5 (Min: 0.1, Max: 2.5)
- sepal length (cm) - 6.0 (Min: 4.3, Max: 7.9)
- sepal width (cm) - 2.9 (Min: 2.0, Max: 4.4)

Instância 4:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal length (cm) - 4.8 (Min: 1.0, Max: 6.

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd

# Função para transformar o dataset em um problema binário (classe 0 contra as outras)
def transformar_problema_binario(y, classe_pos):
    return [1 if label == classe_pos else 0 for label in y]

def analisar_instancias(X, y, class_names, classe_pos=0, instancia_para_analisar=None):
    if not isinstance(X, pd.DataFrame):
        # Adiciona nomes temporários de colunas se X for um array
        X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])

    # Transforma o problema em binário
    y_binario = transformar_problema_binario(y, classe_pos)

    # Divide o dataset em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y_binario, test_size=0.2, random_state=42)
    
    # Treina o modelo
    modelo = LogisticRegression(max_iter=200)
    modelo.fit(X_train, y_train)

    # Obtém os nomes das features
    feature_names = X.columns.tolist()  
    
    # Seleciona as instâncias para análise
    num_instancias = len(X_test)
    instancias_para_analisar = range(num_instancias) if instancia_para_analisar is None else [instancia_para_analisar]
    
    # Loop para analisar cada instância selecionada
    for idx in instancias_para_analisar:
        Vs = X_test.iloc[idx].to_dict()  # Valores da instância específica
        instancia_test = X_test.iloc[[idx]]  # Mantém o formato DataFrame

        # Calcula `gamma_A` usando `decision_function`
        gamma_A = modelo.decision_function(instancia_test)[0]
        
        # Cálculo do valor delta para cada feature
        delta = []
        w = modelo.coef_[0]
        for i, feature in enumerate(feature_names):
            if w[i] < 0:
                delta.append((Vs[feature] - X[feature].max()) * w[i])
            else:
                delta.append((Vs[feature] - X[feature].min()) * w[i])

        # Calcula R
        R = sum(delta) - gamma_A
        
        # Computa a PI-explicação para a instância atual usando nomes das features
        Xpl = one_explanation(Vs, delta, R, feature_names)

        # Imprime os resultados
        classe_verdadeira = y_test[idx]
        print(f"\nInstância {idx}:")
        print(f"Classe verdadeira (binária): {classe_verdadeira}")
        print(f"PI-Explicação: ")
        for item in Xpl:
            print(f"- {item}")
        if not Xpl:
            print('_No-PI-explanation_'*3)
        
    
# Função para calcular a PI-explicação, utilizando nomes das features
def one_explanation(Vs, delta, R, feature_names):
    Xpl = []  # Inicializa a lista de PI-explicação
    delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True)
    R_atual = R  # Inicializa o limiar atual
    Idx = 0  # Inicializa o índice para iterar
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]  # Índice e valor do delta
        feature = feature_names[sorted_idx]  # Nome da característica correspondente
        feature_value = Vs[feature]  # Valor da característica na instância Vs
        min_val = X[feature].min()
        max_val = X[feature].max()

        Xpl.append(f"{feature} - {feature_value}")
        R_atual -= delta_value  # Atualiza o limiar atual
        Idx += 1
    
    return Xpl


# Executar a análise de PI-explicação no conjunto de teste do dataset selecionado
# Certifique-se de que X seja um DataFrame com colunas nomeadas
analisar_instancias(X, y, class_names, classe_pos=0)



Instância 0:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal length (cm) - 4.7
- petal width (cm) - 1.2
- sepal length (cm) - 6.1
- sepal width (cm) - 2.8

Instância 1:
Classe verdadeira (binária): 1
PI-Explicação: 
- petal length (cm) - 1.7
- petal width (cm) - 0.3

Instância 2:
Classe verdadeira (binária): 0
PI-Explicação: 
- sepal width (cm) - 2.6
- petal width (cm) - 2.3
- sepal length (cm) - 7.7
- petal length (cm) - 6.9

Instância 3:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal length (cm) - 4.5
- petal width (cm) - 1.5
- sepal length (cm) - 6.0
- sepal width (cm) - 2.9

Instância 4:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal length (cm) - 4.8
- petal width (cm) - 1.4
- sepal width (cm) - 2.8
- sepal length (cm) - 6.8

Instância 5:
Classe verdadeira (binária): 1
PI-Explicação: 
- petal length (cm) - 1.5
- petal width (cm) - 0.4

Instância 6:
Classe verdadeira (binária): 0
PI-Explicação: 
- petal length (cm) - 3.6
- petal width (cm) - 1.3
- sepal leng